In [2]:
from urllib.request import urlopen
import json
import pandas as pd
import re
from collections import defaultdict

### Differences Between Two BLAST API Call Styles

#### `...?name=2004ef&format=json`

Used more for a general search and a light weight summary of your serached for transient.

- **Purpose:** Search/filter endpoint — you can pass criteria like `name`, `ra`, `dec`, etc., and it returns **a list** (even if there’s just one match).
- **Response type:** JSON array of transient dicts. You’d need to index `[0]` to get the first match.

#### `/api/transient/get/2004ef?format=json`

Returns the Full Science Payload according to the "Science payload Data Model" found in the following link:

https://blast.readthedocs.io/en/stable/usage/web_api.html#api-all

- **Response type:** A single JSON dict (no array).


In [4]:
# Fetch the transient
response = urlopen('https://blast.ncsa.illinois.edu/api/transient/?name=2004ef&format=json')
data = json.loads(response.read())

transient = data[0]  # unpack the first (and only) result

transient.keys()

print("\n--- Transient Information ---\n")

for key, value in transient.items():
    print(f"{key:30}: {value}")

print("\n--- Host Information ---\n")

for key, value in transient['host'].items():
    print(f"{key:30}: {value}")

print(len(transient.keys()), "keys in transient")


--- Transient Information ---

id                            : 2624
ra_deg                        : 340.54174583333327
dec_deg                       : 19.994555555555557
name                          : 2004ef
public_timestamp              : None
redshift                      : 0.03002
spectroscopic_class           : SN Ia
milkyway_dust_reddening       : 0.047228389531373975
image_trim_status             : processed
progress                      : 100
software_version              : 1.2.0
host                          : {'id': 12156, 'ra_deg': 340.54385948, 'dec_deg': 19.9969434, 'name': '2004ef', 'redshift': 0.030988, 'redshift_err': None, 'photometric_redshift': 0.01854147685274371, 'photometric_redshift_err': None, 'milkyway_dust_reddening': 0.04736369989812374, 'software_version': None}

--- Host Information ---

id                            : 12156
ra_deg                        : 340.54385948
dec_deg                       : 19.9969434
name                          : 2004ef
redshi

In [5]:
url = 'https://blast.ncsa.illinois.edu/api/transient/get/2004ef?format=json'
payload = json.loads(urlopen(url).read())  # ← just one dict
print(type(payload)) 
print(len(payload.keys()), "keys in payload")
for key, value in payload.items():
    print(f"{key:30}: {value}")

<class 'dict'>
621 keys in payload
transient_id                  : 2624
transient_ra_deg              : 340.54174583333327
transient_dec_deg             : 19.994555555555557
transient_name                : 2004ef
transient_public_timestamp    : None
transient_redshift            : 0.03002
transient_spectroscopic_class : SN Ia
transient_milkyway_dust_reddening: 0.047228389531373975
transient_image_trim_status   : processed
transient_progress            : 100
transient_software_version    : 1.2.0
transient_host                : {'id': 12156, 'ra_deg': 340.54385948, 'dec_deg': 19.9969434, 'name': '2004ef', 'redshift': 0.030988, 'redshift_err': None, 'photometric_redshift': 0.01854147685274371, 'photometric_redshift_err': None, 'milkyway_dust_reddening': 0.04736369989812374, 'software_version': None}
host_name                     : 2004ef
host_ra_deg                   : 340.54385948
host_dec_deg                  : 19.9969434
host_redshift                 : 0.030988
host_milkyway_dust_redde

In [ ]:
def get_blast_info(sn_name):
    """
    Fetch information for a supernova/transient from the BLAST API.

    Parameters
    ----------
    sn_name : str
        The supernova/transient name (e.g. "2004ef").

    Returns
    -------
    dict
        A dictionary of BLAST fields for the transient.
    """

    # Keep track of what could not be found in the BLAST Query
    missing_from_blast = []

    url = f"https://blast.ncsa.illinois.edu/api/transient/get/{sn_name}?format=json"

    try:
        with urlopen(url) as r:
            payload = json.loads(r.read())

    ## The following catches any Python Exception that happened within the "try" block.
    ## Error object gets stored as the variable e.
    ## Also keeps track of missing supernova names.
    except Exception as e:
        print(f"Error fetching {sn_name}: {e}")
        missing_from_blast.append(sn_name)   # keep track of failures
        return {}

    row = {
        "blast_transient_spectroscopic_class": (payload.get("transient_spectroscopic_class") or "").strip(),
        "blast_transient_photometric_class": (payload.get("transient_photometric_class") or "").strip(),
        
        "blast_host_name": payload.get("host_name"),
        "blast_host_ra_deg": payload.get("host_ra_deg"),
        "blast_host_dec_deg": payload.get("host_dec_deg"),
        "blast_host_redshift": payload.get("host_redshift"),
        "blast_host_EBV_MW": payload.get("host_milkyway_dust_reddening"),

        # Local SED
        "blast_local_log_mass_50": payload.get("local_aperture_host_log_mass_50"),
        "blast_local_log_sfr_50": payload.get("local_aperture_host_log_sfr_50"),
        "blast_local_log_ssfr_50": payload.get("local_aperture_host_log_ssfr_50"),
        "blast_local_log_age_50": payload.get("local_aperture_host_log_age_50"),
        "blast_local_mass_surviving_ratio": payload.get("local_aperture_host_mass_surviving_ratio"),

        # Global SED
        "blast_global_log_mass_50": payload.get("global_aperture_host_log_mass_50"),
        "blast_global_log_sfr_50": payload.get("global_aperture_host_log_sfr_50"),
        "blast_global_log_ssfr_50": payload.get("global_aperture_host_log_ssfr_50"),
        "blast_global_log_age_50": payload.get("global_aperture_host_log_age_50"),
        "blast_global_mass_surviving_ratio": payload.get("global_aperture_host_mass_surviving_ratio"),
    }
    return row

### DES5YR Host Galaxy Association ###

The pirmary challenge of this data 

In [ ]:
# Load your CSV
csv_path = "des_smaple_SNIa.csv"
df = pd.read_csv(csv_path)

missing_from_blast = []


# Apply to each SN in your CSV (assuming column is "CID" or "SNID")
blast_rows = []
for sn in df["CID"]:   # change to df["SNID"] if that's the column name
    blast_rows.append(get_blast_info(sn))

# Merge the results back into your DataFrame
blast_df = pd.DataFrame(blast_rows)
df = pd.concat([df, blast_df], axis=1)

# Save to new file
df.to_csv("explore_webapi_with_blast.csv", index=False)
print(missing_from_blast)

## Taking the Data that We Need

For our purposes, we are interested in a specific subset of BLAST-computed full payload parameters.


### Host SED Fitting Results

- An **SED** is the distribution of a galaxy’s brightness as a function of wavelength (essentially, how much light it emits in each filter).
- Different stellar populations, star formation histories, and dust contents shape the SED differently.
- By comparing the observed photometry (points in different filters) to a grid or model of theoretical SEDs, we can infer underlying galaxy properties.

- BLAST uses the **Prospector** Bayesian inference framework, accelerated with **simulation-based inference**.
- Observed photometry (from surveys like SDSS, PS1, 2MASS, WISE, etc.) is fed into Prospector models.
- A **Bayesian posterior distribution** is generated for each physical parameter.
- The API exposes **summary statistics** from those posteriors: the 16th, 50th (median), and 84th percentiles.

- `<aperture_type>`:
  - **`local`** — properties measured within ~2 kpc of the SN location.
  - **`global`** — properties measured for the entire host galaxy.

- `<parameter>`:
  - **`log_mass`** — log₁₀ of the surviving host stellar mass [M☉]
  - **`log_sfr`** — log₁₀ of the host star formation rate [M☉ yr⁻¹]
  - **`log_ssfr`** — log₁₀ of the host specific star formation rate [yr⁻¹]
  - **`log_age`** — log₁₀ of the mass-weighted stellar age [yr]

NOTE: BLAST provides the 16th, 50th, and 84th percentiles of the posterior for each parameter.


### Host Photometry

We also want the raw host photometry (magnitudes, fluxes, errors) for both local and global apertures,  
as this can be used for further derived quantities (e.g., colors, surface densities). This is just in case we have the time for this.

### Supernova (SN Ia) Information

We would also like to store the following SN level data:

- `sn_name`
- `sn_ra_deg` — SN RA in decimal degrees
- `sn_dec_deg` — SN Dec in decimal degrees
- `sn_redshift` — SN redshift
- `sn_spectroscopic_class`
- `sn_photometric_class`
- `sn_processing_status` — BLAST processing state for this transient

In [ ]:
# Builds the full science payload URL for the given transient name (e.g., 2004ef).
# Opens the URL, reads the response, and parses it from JSON text into a Python dict.

## note: {} inside an f-string ends up being replaced with the value of the variable. 
## the physical charcters of "{" or "}" are not inlcuded. 
def fetch_payload(name: str):
    url = f"https://blast.ncsa.illinois.edu/api/transient/get/{name}?format=json"
    with urlopen(url) as r:
        return json.loads(r.read())


## MAYBE THIS IS UNECESSARY JUST RETURN THE TRANSIENT SPECTROSCOPIC CLASS OR THE TRANSEINT PHOTOMETRICK CASS

## RED FLAG UNNECESSARY AS ALL THE SNIA that we are passing through are already TYPE Ia. Now weathe they are photometrically or spectroscopically confirmed is another issue alltogether.

def is_snia(payload: dict) -> bool:
    """Return True if BLAST thinks this is (probably) a Type Ia."""
    sc = (payload.get("transient_spectroscopic_class") or "").strip().lower()
    pc = (payload.get("transient_photometric_class") or "").strip().lower()
    return ("ia" in sc) or ("ia" in pc)

def _get(payload, key, default=None):
    return payload.get(key, default)

## DEFINES THE PARAMETERS YOU WANT FROM THE PAYLOAD ##
## payload is the input variable and its type is a dictionary. 
def extract_catalog_row(payload: dict):

    ## Create a dictionary called "row". 
    ## values like "transient_name" will be assigned to the left most
    ## keys like "transient_name" on the very left hand side. 
    row = {
        # SN id / position / z / classes
        "transient_name": _get(payload, "transient_name"),
        "transient_spectroscopic_class": (payload.get("transient_spectroscopic_class") or "").strip(),
        "transient_photometric_class": (payload.get("transient_photometric_class") or "").strip(),
        "transient_ra_deg": _get(payload, "transient_ra_deg"),
        "transient_dec_deg": _get(payload, "transient_dec_deg"),
        "transient_redshift": _get(payload, "transient_redshift"),
        "transient_spectroscopic_class": (payload.get("transient_spectroscopic_class") or "").strip(),
        "transient_photometric_class": (payload.get("transient_photometric_class") or "").strip(),
        "transient_processing_status": _get(payload, "transient_processing_status"),

        # Host basics
        "host_name": _get(payload, "host_name"),
        "host_ra_deg": _get(payload, "host_ra_deg"),
        "host_dec_deg": _get(payload, "host_dec_deg"),
        "host_redshift": _get(payload, "host_redshift"),
        "host_EBV_MW": _get(payload, "host_milkyway_dust_reddening"),
    }

    # SED medians (p50) for both local & global ( p16/p84 are not included)
    params = ["log_mass", "log_sfr", "log_ssfr", "log_age"]
    for scope in ("local", "global"):
        for p in params:
            key = f"{scope}_aperture_host_{p}_50"
            row[f"{scope}_{p}_50"] = _get(payload, key)
        # optional quality-ish extra
        row[f"{scope}_mass_surviving_ratio"] = _get(payload, f"{scope}_aperture_host_mass_surviving_ratio")

    return row

_PHOT_RE = re.compile(
    r'^(?P<scope>local|global)_aperture_(?P<survey>[A-Za-z0-9]+)_(?P<band>[A-Za-z0-9]+)_(?P<quantity>magnitude|magnitude_error|flux|flux_error|is_validated)$',
    re.I
)

def extract_photometry_rows(payload: dict):
    """Return long-form rows for photometry with mag/flux + errors."""
    rows = defaultdict(dict)
    base_id = {
        "sn_name": _get(payload, "transient_name"),
        "host_name": _get(payload, "host_name"),
    }

    for k, v in payload.items():
        m = _PHOT_RE.match(k)
        if not m:
            continue
        scope, survey, band, quantity = m.group("scope","survey","band","quantity")
        idx = (scope.lower(), survey, band)
        if "scope" not in rows[idx]:
            rows[idx].update(base_id)
            rows[idx]["scope"] = scope.lower()
            rows[idx]["survey"] = survey
            rows[idx]["band"] = band
        rows[idx][quantity] = v

    # normalize booleans, missing values
    out = []
    for r in rows.values():
        r.setdefault("magnitude", None)
        r.setdefault("magnitude_error", None)
        r.setdefault("flux", None)
        r.setdefault("flux_error", None)
        # convert is_validated to bool if present
        if "is_validated" in r and isinstance(r["is_validated"], str):
            r["is_validated"] = r["is_validated"].lower() == "true"
        out.append(r)
    return out

# --------------------------- main routine ---------------------------

def build_blash_catalog(sn_names, require_snia=True,
                        catalog_csv="blast_hosts_catalog.csv",
                        photometry_csv="blast_hosts_photometry.csv"):
    cat_rows, phot_rows = [], []

    for name in sn_names:
        try:
            payload = fetch_payload(name)
        except Exception as e:
            print(f"[WARN] {name}: fetch failed: {e}")
            continue

        if require_snia and not is_snia(payload):
            print(f"[SKIP] {name}: not Ia by BLAST labels")
            continue

        cat_rows.append(extract_catalog_row(payload))
        phot_rows.extend(extract_photometry_rows(payload))

    cat_df = pd.DataFrame(cat_rows)
    phot_df = pd.DataFrame(phot_rows)

    # Save
    if not cat_df.empty:
        cat_df.to_csv(catalog_csv, index=False)
        print(f"[OK] wrote {catalog_csv} ({len(cat_df)} rows)")
    else:
        print("[INFO] catalog is empty")

    if not phot_df.empty:
        phot_df.sort_values(["sn_name","scope","survey","band"]).to_csv(photometry_csv, index=False)
        print(f"[OK] wrote {photometry_csv} ({len(phot_df)} rows)")
    else:
        print("[INFO] photometry is empty")

    return cat_df, phot_df

# --------------------------- example --------------------------- #

if __name__ == "__main__":
    # Replace with your list
    targets = ["2004ef", "2018gv", "2011fe", "2014J"]
    catalog, phot = build_blash_catalog(targets)

[OK] wrote blast_hosts_catalog.csv (4 rows)
[OK] wrote blast_hosts_photometry.csv (192 rows)


## Output Analysis

When we perform a **full payload** BLAST API query (`/api/transient/get/<name>?format=json`),  
the returned JSON is large and contains a wide range of information.

For our purposes, we are interested in a **specific subset** of BLAST-computed parameters.


### SED Fitting Results

**API reference:** `/api/sedfittingresult/`

- `<aperture_type>` can be either:
  - **`local`** — properties measured within ~2 kpc of the SN location.
  - **`global`** — properties measured for the entire host galaxy.

- `<parameter>` can be one of:
  - **`log_mass`** — log₁₀ of the surviving host stellar mass [M☉]
  - **`log_sfr`** — log₁₀ of the host star formation rate [M☉ yr⁻¹]
  - **`log_ssfr`** — log₁₀ of the host specific star formation rate [yr⁻¹]
  - **`log_age`** — log₁₀ of the mass-weighted stellar age [yr]

BLAST typically provides the **16th, 50th, and 84th percentiles** of the posterior for each parameter.


### Host Photometry

We also want the **host photometry** (magnitudes, fluxes, errors) for both local and global apertures,  
as this can be used for further derived quantities (e.g., colors, surface densities).


### Supernova (SN Ia) Information

If available in the BLAST payload, we also store the following SN-level metadata:

- `sn_name`
- `sn_ra_deg` — SN RA in decimal degrees
- `sn_dec_deg` — SN Dec in decimal degrees
- `sn_redshift` — SN redshift
- `sn_spectroscopic_class`
- `sn_photometric_class`
- `sn_processing_status` — BLAST processing state for this transient

In [9]:
# Load the CSV
df = pd.read_csv("blast_hosts_catalog.csv")

# Show the column names
print("Number of columns:", len(df.columns))
print("\nColumn names:")
for col in df.columns:
    print(f"- {col}")

Number of columns: 22

Column names:
- sn_name
- sn_ra_deg
- sn_dec_deg
- sn_redshift
- sn_spectroscopic_class
- sn_photometric_class
- sn_processing_status
- host_name
- host_ra_deg
- host_dec_deg
- host_redshift
- host_EBV_MW
- local_log_mass_50
- local_log_sfr_50
- local_log_ssfr_50
- local_log_age_50
- local_mass_surviving_ratio
- global_log_mass_50
- global_log_sfr_50
- global_log_ssfr_50
- global_log_age_50
- global_mass_surviving_ratio


# What BLAST Gives You for **Host Galaxies** (useful fields to pull)

BLAST pre-computes host properties by fitting multi-band photometry with **Prospector** (Bayesian SED inference, accelerated via simulation-based inference). You get values for both **global** (whole galaxy) and **local** (≈2 kpc around the SN) apertures.

The complete list of the data being stored can be found at the following 

## 1) Host identification & basics
- `host_name` — host identifier (may be internal)
- `host_ra_deg`, `host_dec_deg` — host coordinates (deg)
- `host_redshift` — spectroscopic/assigned host redshift (dimensionless)
- `host_milkyway_dust_reddening` — Galactic E(B−V) toward the host (mag)

> Also useful for context:
> - `sn_name`, `sn_ra_deg`, `sn_dec_deg`, `sn_redshift`
> - `sn_spectroscopic_class`, `sn_photometric_class`

## 3) SED-fit **derived** physical properties (per scope)
BLAST exposes percentile summaries (typically 16/50/84). Field-name pattern:
`<scope>_aperture_host_<parameter>_<percentile>`

Core parameters:
- `..._log_mass_{16,50,84}` — log10 stellar mass (M☉)
- `..._log_sfr_{16,50,84}` — log10 SFR (M☉/yr)
- `..._log_ssfr_{16,50,84}` — log10 sSFR (yr⁻¹)
- `..._log_age_{16,50,84}` — log10 mass-weighted stellar age (yr)
- `..._mass_surviving_ratio` — surviving / formed stellar mass (dimensionless)

> Notes
> - The “50” (median) is what your current CSV keeps (e.g., `local_log_mass_50`).
> - Keep 16/84 as uncertainties: e.g., `log_mass_err_minus = 50−16`, `log_mass_err_plus = 84−50`.
> - Some payloads may include additional SED parameters (e.g., `log_tau`) depending on the BLAST run.

## 4) **Photometry** used by the SED fits (per scope, survey, and band)
Field-name pattern:
`<scope>_aperture_<SURVEY>_<BAND>_{magnitude,magnitude_error,flux,flux_error,is_validated}`

Examples (not exhaustive):
- `local_aperture_SDSS_g_magnitude`, `local_aperture_SDSS_g_magnitude_error`
- `global_aperture_PS1_i_flux`, `global_aperture_PS1_i_flux_error`
- `..._is_validated` (boolean-ish flag)

> Use these if you want to recompute colors, surface densities, or test alternative SED assumptions.

---

## Quick reference: what to keep in a tidy “host catalog” table
For each SN (row), include at least:

**Identification**
- `sn_name`, `host_name`, `host_redshift`, `host_ra_deg`, `host_dec_deg`, `host_milkyway_dust_reddening`

**Global (whole galaxy)**
- `global_log_mass_50`, `global_log_mass_16`, `global_log_mass_84`
- `global_log_sfr_50`,  `global_log_sfr_16`,  `global_log_sfr_84`
- `global_log_ssfr_50`, `global_log_ssfr_16`, `global_log_ssfr_84`
- `global_log_age_50`,  `global_log_age_16`,  `global_log_age_84`
- `global_mass_surviving_ratio`

**Local (≈2 kpc)**
- `local_log_mass_50`, `local_log_mass_16`, `local_log_mass_84`
- `local_log_sfr_50`,  `local_log_sfr_16`,  `local_log_sfr_84`
- `local_log_ssfr_50`, `local_log_ssfr_16`, `local_log_ssfr_84`
- `local_log_age_50`,  `local_log_age_16`,  `local_log_age_84`
- `local_mass_surviving_ratio`

---

## Optional “you compute it” derived quantities (from the above)
- **Uncertainty widths**: e.g., `σ⁺ = p84 − p50`, `σ⁻ = p50 − p16`
- **Surface stellar mass density** (Σ\*) if aperture areas are known
- **Colors** from photometry (e.g., `g−r`, `r−i`) for sanity checks
- **Δ(local−global)** contrasts (e.g., Δlog sSFR) for environment studies

---

### Units & conventions
- Angles in **degrees**
- Redshifts dimensionless
- E(B−V) in **magnitudes**
- `log_mass` in **log10(M☉)**
- `log_sfr` in **log10(M☉/yr)**
- `log_ssfr` in **log10(yr⁻¹)**
- `log_age` in **log10(yr)**
- Percentiles: `{16,50,84}` ≈ (−1σ, median, +1σ) if posteriors are ~Gaussian

In [9]:
from urllib.request import urlopen
import json

def get_ra_from_name(sn_name: str) -> float:
    """
    Fetch BLAST payload for a given SN name and return its RA in degrees.
    """
    url = f"https://blast.ncsa.illinois.edu/api/transient/get/{sn_name}?format=json"
    with urlopen(url) as r:
        payload = json.loads(r.read())
    return payload.get("transient_ra_deg")

# Example
print(get_ra_from_name("2004ef"))

340.54174583333327


In [10]:
from urllib.request import urlopen
import json
import pandas as pd

# Pick one SN
sn_name = "2004ef"

# Fetch payload from BLAST
url = f"https://blast.ncsa.illinois.edu/api/transient/get/{sn_name}?format=json"
with urlopen(url) as r:
    payload = json.loads(r.read())

# Grab just the RA
ra = payload.get("transient_ra_deg")

# Put into a DataFrame
df = pd.DataFrame([{"sn_name": sn_name, "ra_deg": ra}])

# Save to CSV
df.to_csv("single_snia.csv", index=False)

print(df)

  sn_name      ra_deg
0  2004ef  340.541746


In [11]:
from urllib.request import urlopen
import json
import pandas as pd

# Pick one SN
sn_name = "2004ef"

# Fetch payload from BLAST
url = f"https://blast.ncsa.illinois.edu/api/transient/get/{sn_name}?format=json"
with urlopen(url) as r:
    payload = json.loads(r.read())

# Collect all the desired fields
row = {
    "transient_name": payload.get("transient_name"),
    "transient_spectroscopic_class": (payload.get("transient_spectroscopic_class") or "").strip(),
    "transient_photometric_class": (payload.get("transient_photometric_class") or "").strip(),
    "transient_ra_deg": payload.get("transient_ra_deg"),
    "transient_dec_deg": payload.get("transient_dec_deg"),
    "transient_redshift": payload.get("transient_redshift"),
    "transient_processing_status": payload.get("transient_processing_status"),

    "host_name": payload.get("host_name"),
    "host_ra_deg": payload.get("host_ra_deg"),
    "host_dec_deg": payload.get("host_dec_deg"),
    "host_redshift": payload.get("host_redshift"),
    "host_EBV_MW": payload.get("host_milkyway_dust_reddening"),

    # Local SED
    "local_log_mass_50": payload.get("local_aperture_host_log_mass_50"),
    "local_log_sfr_50": payload.get("local_aperture_host_log_sfr_50"),
    "local_log_ssfr_50": payload.get("local_aperture_host_log_ssfr_50"),
    "local_log_age_50": payload.get("local_aperture_host_log_age_50"),
    "local_mass_surviving_ratio": payload.get("local_aperture_host_mass_surviving_ratio"),

    # Global SED
    "global_log_mass_50": payload.get("global_aperture_host_log_mass_50"),
    "global_log_sfr_50": payload.get("global_aperture_host_log_sfr_50"),
    "global_log_ssfr_50": payload.get("global_aperture_host_log_ssfr_50"),
    "global_log_age_50": payload.get("global_aperture_host_log_age_50"),
    "global_mass_surviving_ratio": payload.get("global_aperture_host_mass_surviving_ratio"),
}

# Put into a DataFrame
df = pd.DataFrame([row])

# Save to CSV
df.to_csv("single_snia_full.csv", index=False)

print(df)

  transient_name transient_spectroscopic_class transient_photometric_class  \
0         2004ef                         SN Ia                               

   transient_ra_deg  transient_dec_deg  transient_redshift  \
0        340.541746          19.994556             0.03002   

  transient_processing_status host_name  host_ra_deg  host_dec_deg  ...  \
0                        None    2004ef   340.543859     19.996943  ...   

   local_log_mass_50  local_log_sfr_50  local_log_ssfr_50  local_log_age_50  \
0           9.222703         -1.142086         -10.369217          7.884289   

   local_mass_surviving_ratio  global_log_mass_50  global_log_sfr_50  \
0                     0.57383           10.949363          -0.096142   

   global_log_ssfr_50  global_log_age_50  global_mass_surviving_ratio  
0          -11.008361           8.004165                      0.58122  

[1 rows x 22 columns]


### DES ###

In [16]:
# Load your CSV
csv_path = "des_smaple_SNIa.csv"
df = pd.read_csv(csv_path)

missing_from_blast = []


# Make a function to fetch BLAST info for one SN
def get_blast_info(sn_name):
    url = f"https://blast.ncsa.illinois.edu/api/transient/get/{sn_name}?format=json"
    try:
        with urlopen(url) as r:
            payload = json.loads(r.read())
    except Exception as e:
        print(f"Error fetching {sn_name}: {e}")
        missing_from_blast.append(sn_name)   # keep track of failures
        return {}

    row = {
        "blast_transient_spectroscopic_class": (payload.get("transient_spectroscopic_class") or "").strip(),
        "blast_transient_photometric_class": (payload.get("transient_photometric_class") or "").strip(),
        
        "blast_host_name": payload.get("host_name"),
        "blast_host_ra_deg": payload.get("host_ra_deg"),
        "blast_host_dec_deg": payload.get("host_dec_deg"),
        "blast_host_redshift": payload.get("host_redshift"),
        "blast_host_EBV_MW": payload.get("host_milkyway_dust_reddening"),

        # Local SED
        "blast_local_log_mass_50": payload.get("local_aperture_host_log_mass_50"),
        "blast_local_log_sfr_50": payload.get("local_aperture_host_log_sfr_50"),
        "blast_local_log_ssfr_50": payload.get("local_aperture_host_log_ssfr_50"),
        "blast_local_log_age_50": payload.get("local_aperture_host_log_age_50"),
        "blast_local_mass_surviving_ratio": payload.get("local_aperture_host_mass_surviving_ratio"),

        # Global SED
        "blast_global_log_mass_50": payload.get("global_aperture_host_log_mass_50"),
        "blast_global_log_sfr_50": payload.get("global_aperture_host_log_sfr_50"),
        "blast_global_log_ssfr_50": payload.get("global_aperture_host_log_ssfr_50"),
        "blast_global_log_age_50": payload.get("global_aperture_host_log_age_50"),
        "blast_global_mass_surviving_ratio": payload.get("global_aperture_host_mass_surviving_ratio"),
    }
    return row

# Apply to each SN in your CSV (assuming column is "CID" or "SNID")
blast_rows = []
for sn in df["CID"]:   # change to df["SNID"] if that's the column name
    blast_rows.append(get_blast_info(sn))

# Merge the results back into your DataFrame
blast_df = pd.DataFrame(blast_rows)
df = pd.concat([df, blast_df], axis=1)

# Save to new file
df.to_csv("explore_webapi_with_blast.csv", index=False)
print(missing_from_blast)

Error fetching 2001ah: HTTP Error 404: Not Found
Error fetching 2001ay: HTTP Error 404: Not Found
Error fetching 2001eh: HTTP Error 404: Not Found
Error fetching 2002he: HTTP Error 404: Not Found
Error fetching 2002hu: HTTP Error 404: Not Found
Error fetching 2003fa: HTTP Error 404: Not Found
Error fetching 2003iv: HTTP Error 404: Not Found
Error fetching 2005ms: HTTP Error 404: Not Found
Error fetching 2006S: HTTP Error 404: Not Found
Error fetching 2006az: HTTP Error 404: Not Found
Error fetching 2006cf: HTTP Error 404: Not Found
Error fetching 2006oa: HTTP Error 404: Not Found
Error fetching 2006on: HTTP Error 404: Not Found
Error fetching 2006qo: HTTP Error 404: Not Found
Error fetching 2007R: HTTP Error 404: Not Found
Error fetching 2007sw: HTTP Error 404: Not Found
Error fetching 2008050: HTTP Error 404: Not Found
Error fetching 2008051: HTTP Error 404: Not Found
Error fetching 2010dt: HTTP Error 404: Not Found
Error fetching AT2016ews: HTTP Error 404: Not Found
Error fetching AT

In [17]:
import pandas as pd

csv_in  = "explore_webapi_with_blast.csv"
csv_out = "explore_webapi_with_blast_only.csv"
csv_missing = "missing_from_blast.csv"

# Load
df = pd.read_csv(csv_in)

# All BLAST columns (those we added with the 'blast_' prefix)
blast_cols = [c for c in df.columns if c.startswith("blast_")]

# Mark rows that have at least ONE non-null BLAST value
has_blast = df[blast_cols].notna().any(axis=1)

# Keep only rows with BLAST info
kept = df[has_blast].copy()

# Rows removed = missing from BLAST entirely
missing = df[~has_blast].copy()

# (Optional) pick a likely name/ID column to show in the missing file
name_col = next((c for c in ["CID", "SNID", "transient_name"] if c in df.columns), None)
if name_col:
    missing = missing[[name_col] + blast_cols]  # show identifier + the empty blast columns

# Save results
kept.to_csv(csv_out, index=False)
missing.to_csv(csv_missing, index=False)

print(f"Kept {len(kept)} rows with BLAST data → {csv_out}")
print(f"Removed {len(missing)} rows missing BLAST data → {csv_missing}")

Kept 161 rows with BLAST data → explore_webapi_with_blast_only.csv
Removed 34 rows missing BLAST data → missing_from_blast.csv


## SHOES ###

In [21]:
# Load your CSV
csv_path = "Pantheon+SH0ES.csv"
df = pd.read_csv(csv_path)

missing_from_blast = []


# Make a function to fetch BLAST info for one SN
def get_blast_info(sn_name):
    url = f"https://blast.ncsa.illinois.edu/api/transient/get/{sn_name}?format=json"
    try:
        with urlopen(url) as r:
            payload = json.loads(r.read())
    except Exception as e:
        print(f"Error fetching {sn_name}: {e}")
        missing_from_blast.append(sn_name)   # keep track of failures
        return {}

    row = {
        "blast_transient_spectroscopic_class": (payload.get("transient_spectroscopic_class") or "").strip(),
        "blast_transient_photometric_class": (payload.get("transient_photometric_class") or "").strip(),
        
        "blast_host_name": payload.get("host_name"),
        "blast_host_ra_deg": payload.get("host_ra_deg"),
        "blast_host_dec_deg": payload.get("host_dec_deg"),
        "blast_host_redshift": payload.get("host_redshift"),
        "blast_host_EBV_MW": payload.get("host_milkyway_dust_reddening"),

        # Local SED
        "blast_local_log_mass_50": payload.get("local_aperture_host_log_mass_50"),
        "blast_local_log_sfr_50": payload.get("local_aperture_host_log_sfr_50"),
        "blast_local_log_ssfr_50": payload.get("local_aperture_host_log_ssfr_50"),
        "blast_local_log_age_50": payload.get("local_aperture_host_log_age_50"),
        "blast_local_mass_surviving_ratio": payload.get("local_aperture_host_mass_surviving_ratio"),

        # Global SED
        "blast_global_log_mass_50": payload.get("global_aperture_host_log_mass_50"),
        "blast_global_log_sfr_50": payload.get("global_aperture_host_log_sfr_50"),
        "blast_global_log_ssfr_50": payload.get("global_aperture_host_log_ssfr_50"),
        "blast_global_log_age_50": payload.get("global_aperture_host_log_age_50"),
        "blast_global_mass_surviving_ratio": payload.get("global_aperture_host_mass_surviving_ratio"),
    }
    return row

# Apply to each SN in your CSV (assuming column is "CID" or "SNID")
blast_rows = []
for sn in df["CID"]:   # change to df["SNID"] if that's the column name
    blast_rows.append(get_blast_info(sn))

# Merge the results back into your DataFrame
blast_df = pd.DataFrame(blast_rows)
df = pd.concat([df, blast_df], axis=1)

# Save to new file
df.to_csv("explore_webapi_with_blast_SHOES.csv", index=False)
print(missing_from_blast)

Error fetching 1994DRichmond: HTTP Error 404: Not Found
Error fetching 2017cbv: HTTP Error 404: Not Found
Error fetching 2017cbv: HTTP Error 404: Not Found
Error fetching 1998aq: HTTP Error 404: Not Found
Error fetching 1990N: HTTP Error 404: Not Found
Error fetching 2005df_ANU: HTTP Error 404: Not Found
Error fetching 1992A: HTTP Error 404: Not Found
Error fetching 2016coj: HTTP Error 404: Not Found
Error fetching 2007gi: HTTP Error 404: Not Found
Error fetching 1994ae: HTTP Error 404: Not Found
Error fetching 2015bp: HTTP Error 404: Not Found
Error fetching 1998dm: HTTP Error 404: Not Found
Error fetching 2014bv: HTTP Error 404: Not Found
Error fetching 1997br: HTTP Error 404: Not Found
Error fetching 1995al: HTTP Error 404: Not Found
Error fetching 1996X: HTTP Error 404: Not Found
Error fetching 2017erp: HTTP Error 404: Not Found
Error fetching 2017erp: HTTP Error 404: Not Found
Error fetching 2017erp: HTTP Error 404: Not Found
Error fetching 1992G: HTTP Error 404: Not Found
Error f

In [23]:
import pandas as pd

csv_in  = "explore_webapi_with_blast_SHOES.csv"
csv_out = "explore_webapi_with_blast_only_SHOES.csv"
csv_missing = "missing_from_blast_SHOES.csv"

# Load
df = pd.read_csv(csv_in)

# All BLAST columns (those we added with the 'blast_' prefix)
blast_cols = [c for c in df.columns if c.startswith("blast_")]

# Mark rows that have at least ONE non-null BLAST value
has_blast = df[blast_cols].notna().any(axis=1)

# Keep only rows with BLAST info
kept = df[has_blast].copy()

# Rows removed = missing from BLAST entirely
missing = df[~has_blast].copy()

# (Optional) pick a likely name/ID column to show in the missing file
name_col = next((c for c in ["CID", "SNID", "transient_name"] if c in df.columns), None)
if name_col:
    missing = missing[[name_col] + blast_cols]  # show identifier + the empty blast columns

# Save results
kept.to_csv(csv_out, index=False)
missing.to_csv(csv_missing, index=False)

print(f"Kept {len(kept)} rows with BLAST data → {csv_out}")
print(f"Removed {len(missing)} rows missing BLAST data → {csv_missing}")

Kept 543 rows with BLAST data → explore_webapi_with_blast_only_SHOES.csv
Removed 1158 rows missing BLAST data → missing_from_blast_SHOES.csv


## ZTF ##

In [20]:
# Load your CSV
csv_path = "ZTF_snia_data.csv"
df = pd.read_csv(csv_path)

missing_from_blast = []


# Make a function to fetch BLAST info for one SN
def get_blast_info(sn_name):
    url = f"https://blast.ncsa.illinois.edu/api/transient/get/{sn_name}?format=json"
    try:
        with urlopen(url) as r:
            payload = json.loads(r.read())
    except Exception as e:
        print(f"Error fetching {sn_name}: {e}")
        missing_from_blast.append(sn_name)   # keep track of failures
        return {}

    row = {
        "blast_transient_spectroscopic_class": (payload.get("transient_spectroscopic_class") or "").strip(),
        "blast_transient_photometric_class": (payload.get("transient_photometric_class") or "").strip(),
        
        "blast_host_name": payload.get("host_name"),
        "blast_host_ra_deg": payload.get("host_ra_deg"),
        "blast_host_dec_deg": payload.get("host_dec_deg"),
        "blast_host_redshift": payload.get("host_redshift"),
        "blast_host_EBV_MW": payload.get("host_milkyway_dust_reddening"),

        # Local SED
        "blast_local_log_mass_50": payload.get("local_aperture_host_log_mass_50"),
        "blast_local_log_sfr_50": payload.get("local_aperture_host_log_sfr_50"),
        "blast_local_log_ssfr_50": payload.get("local_aperture_host_log_ssfr_50"),
        "blast_local_log_age_50": payload.get("local_aperture_host_log_age_50"),
        "blast_local_mass_surviving_ratio": payload.get("local_aperture_host_mass_surviving_ratio"),

        # Global SED
        "blast_global_log_mass_50": payload.get("global_aperture_host_log_mass_50"),
        "blast_global_log_sfr_50": payload.get("global_aperture_host_log_sfr_50"),
        "blast_global_log_ssfr_50": payload.get("global_aperture_host_log_ssfr_50"),
        "blast_global_log_age_50": payload.get("global_aperture_host_log_age_50"),
        "blast_global_mass_surviving_ratio": payload.get("global_aperture_host_mass_surviving_ratio"),
    }
    return row

# Apply to each SN in your CSV (assuming column is "CID" or "SNID")
blast_rows = []
for sn in df["iau_name"]:
    blast_rows.append(get_blast_info(sn))

# Merge the results back into your DataFrame
blast_df = pd.DataFrame(blast_rows)
df = pd.concat([df, blast_df], axis=1)

# Save to new file
df.to_csv("explore_webapi_with_blast_SHOES.csv", index=False)
print(missing_from_blast)

Error fetching 2018lq: HTTP Error 404: Not Found
Error fetching 2018kc: HTTP Error 404: Not Found
Error fetching 2018xi: HTTP Error 404: Not Found
Error fetching 2020chl: HTTP Error 404: Not Found
Error fetching 2018aey: HTTP Error 404: Not Found
Error fetching 2018aic: HTTP Error 404: Not Found
Error fetching 2018abj: HTTP Error 404: Not Found
Error fetching nan: HTTP Error 404: Not Found
Error fetching 2018aii: HTTP Error 404: Not Found
Error fetching 2018aab: HTTP Error 404: Not Found
Error fetching 2020biz: HTTP Error 404: Not Found
Error fetching 2019abh: HTTP Error 404: Not Found
Error fetching 2018aej: HTTP Error 404: Not Found
Error fetching nan: HTTP Error 404: Not Found
Error fetching 2018zp: HTTP Error 404: Not Found
Error fetching 2018bav: HTTP Error 404: Not Found
Error fetching 2018bat: HTTP Error 404: Not Found
Error fetching 2018aqy: HTTP Error 404: Not Found
Error fetching 2018loy: HTTP Error 404: Not Found
Error fetching 2018ast: HTTP Error 404: Not Found
Error fetchi

In [22]:
import pandas as pd

csv_in  = "explore_webapi_with_blast_ZTF.csv"
csv_out = "explore_webapi_with_blast_only_ZTF.csv"
csv_missing = "missing_from_blast_ZTF.csv"

# Load
df = pd.read_csv(csv_in)

# All BLAST columns (those we added with the 'blast_' prefix)
blast_cols = [c for c in df.columns if c.startswith("blast_")]

# Mark rows that have at least ONE non-null BLAST value
has_blast = df[blast_cols].notna().any(axis=1)

# Keep only rows with BLAST info
kept = df[has_blast].copy()

# Rows removed = missing from BLAST entirely
missing = df[~has_blast].copy()

# (Optional) pick a likely name/ID column to show in the missing file
name_col = next((c for c in ["CID", "SNID", "transient_name"] if c in df.columns), None)
if name_col:
    missing = missing[[name_col] + blast_cols]  # show identifier + the empty blast columns

# Save results
kept.to_csv(csv_out, index=False)
missing.to_csv(csv_missing, index=False)

print(f"Kept {len(kept)} rows with BLAST data → {csv_out}")
print(f"Removed {len(missing)} rows missing BLAST data → {csv_missing}")

Kept 1018 rows with BLAST data → explore_webapi_with_blast_only_ZTF.csv
Removed 2610 rows missing BLAST data → missing_from_blast_ZTF.csv
